<a href="https://colab.research.google.com/github/hyewwn/COSE474/blob/main/CLIP_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# pip install clip

In [1]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00


In [2]:
# pip install openai-clip

In [3]:
pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-r3i7ud8j
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-r3i7ud8j
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 808.7 kB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369497 sha256=02400187b635684880decef421c144269991d31f1e97eb25c9b4211ec8cfc76d
  Stored in directory: /tmp/pip-ephem-wheel-cache-3bj3e17f/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [29]:
from transformers import CLIPModel, CLIPTokenizer, CLIPProcessor
from datasets import load_dataset, load_metric
from torch.optim import AdamW
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import Image
import requests
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import clip
from datasets import load_dataset, DatasetDict

In [5]:
# Load the dataset
dataset = load_dataset("JotDe/mscoco_15k")


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/9988 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4994 [00:00<?, ? examples/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'text'],
        num_rows: 9988
    })
    test: Dataset({
        features: ['image', 'text'],
        num_rows: 4994
    })
})

In [30]:
small_train_dataset = dataset['train'].shuffle(seed=42).select([i for i in range(900)])
small_test_dataset = dataset['test'].shuffle(seed=42).select([i for i in range(400)])

# 새로운 DatasetDict 생성
small_dataset_dict = DatasetDict({
    'train': small_train_dataset,
    'test': small_test_dataset
})

In [7]:
# Load the CLIP model and tokenizer
model_name = "openai/clip-vit-base-patch32"
model = CLIPModel.from_pretrained(model_name)
tokenizer = CLIPTokenizer.from_pretrained(model_name)
processor = CLIPProcessor.from_pretrained(model_name)

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [9]:
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)

100%|████████████████████████████████████████| 338M/338M [00:02<00:00, 131MiB/s]


In [31]:
# 가상의 Dataset 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data
        # self.transform = transforms.PILToTensor()


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = preprocess(self.data[idx]['image'])
        text = self.data[idx]['text']
        # tensor_image = self.transform(image)
        # input_dict = {'tensor_image': image, 'text': text}

        return image, text

# 가상의 데이터셋 생성
dataset_dict = {
    "train": CustomDataset(small_dataset_dict['train']),  # data_train은 실제 데이터를 담고 있는 리스트 또는 객체여야 합니다.
    "test": CustomDataset(small_dataset_dict['test'])     # data_test은 실제 데이터를 담고 있는 리스트 또는 객체여야 합니다.
}

# DataLoader 파라미터 정의
batch_size = 100
shuffle = True

# DataLoader 생성
dataloaders = {split: DataLoader(dataset, batch_size=batch_size, shuffle=shuffle) for split, dataset in dataset_dict.items()}


In [32]:
trainloader = dataloaders['train']
testloader = dataloaders['test']

In [33]:
# Function to convert model's parameters to FP32 format
def convert_models_to_fp32(model):
    for p in model.parameters():
        p.data = p.data.float()
        p.grad.data = p.grad.data.float()

In [34]:
if device == "cpu":
  model.float()

In [35]:
from torch.nn import DataParallel

In [36]:
if device != "cpu":
    model = DataParallel(model)

In [37]:
trainloader

In [38]:
cd /content/drive/MyDrive/딥러닝/project

/content/drive/MyDrive/딥러닝/project


In [39]:
from torch.nn.functional import log_softmax
from nltk.translate.bleu_score import corpus_bleu

# Prepare the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2)
# Specify the loss function
loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()

best_te_loss = 1e5
best_ep = -1
# Train the model
num_epochs = 3
for epoch in range(num_epochs):
  print(f"running epoch {epoch}, best test loss {best_te_loss} after epoch {best_ep}")
  model.train()
  tr_loss = 0
  step = 0
  pbar = tqdm(trainloader, total=len(trainloader))
  for batch in pbar:
    step += 1
    optimizer.zero_grad()

    images,texts = batch

    images= images.to(device)
    texts = clip.tokenize(texts).to(device)

    # Forward pass
    logits_per_image,logits_per_text  = model(images, texts)


    # Compute loss
    ground_truth = torch.arange(len(images),dtype=torch.long,device=device)
    total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
    print(total_loss)

    # Backward pass
    total_loss.backward()
    tr_loss += total_loss.item()
    if device == "cpu":
      optimizer.step()
    else :
      convert_models_to_fp32(model)
      optimizer.step()
      clip.model.convert_weights(model)

    pbar.set_description(f"Epoch {epoch}/{num_epochs}, Loss: {total_loss.item():.4f}", refresh=True)
  tr_loss /= step

  # Eval
  step = 0
  te_loss = 0
  with torch.no_grad():
    model.eval()
    test_pbar = tqdm(testloader, total=len(testloader))
    for batch in test_pbar:
      step += 1
      images,texts = batch
      images= images.to(device)
      texts = clip.tokenize(texts).to(device)
      logits_per_image,logits_per_text  = model(images, texts)
      ground_truth = torch.arange(len(images),dtype=torch.long,device=device)

      total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
      te_loss += total_loss.item()
      test_pbar.set_description(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss.item():.4f}", refresh=True)
    te_loss /= step

  # Save the model if the test loss is the best we've seen so far.
  if te_loss < best_te_loss:
    best_te_loss = te_loss
    best_ep = epoch
    torch.save(model.state_dict(), "test2_best_model.pt")
    print(f"epoch {epoch}, tr_loss {tr_loss}, te_loss {te_loss}")
torch.save(model.state_dict(), "test2_last_model.pt")

running epoch 0, best test loss 100000.0 after epoch -1


  0%|          | 0/9 [00:00<?, ?it/s]

tensor(0.6655, device='cuda:0', dtype=torch.float16, grad_fn=<DivBackward0>)


Epoch 0/3, Loss: 0.6655:  11%|█         | 1/9 [00:09<01:13,  9.19s/it]

tensor(0.9209, device='cuda:0', dtype=torch.float16, grad_fn=<DivBackward0>)


...

tensor(0.3413, device='cuda:0', dtype=torch.float16, grad_fn=<DivBackward0>)


Epoch 2/3, Loss: 0.3413:  89%|████████▉ | 8/9 [00:08<00:01,  1.07s/it]

tensor(0.1309, device='cuda:0', dtype=torch.float16, grad_fn=<DivBackward0>)


Epoch 2/3, Loss: 0.8877: 100%|██████████| 4/4 [00:04<00:00,  1.09s/it]


epoch 2, tr_loss 0.22330729166666666, te_loss 0.9432373046875
